In [113]:
from __future__ import print_function
from music21 import *
from collections import Counter, defaultdict
from ea.individual import Measure, Individual, Note
from ea.duration import Duration

import ea.initialisation as initialisation
import ea.modelTrainer as modelTrainer
import ea.musicPlayer as musicPlayer
import ea.simulation as simulation
import ea.fitness as fitness
import collections
import importlib
import music21
import numpy as np
import pandas as pd
import ea.individual as individual
import ea.duration as duration
import ea.constants as constants
import ea.util as util
import random
import nltk
import ea.fitness as fitness
import re
import ea.modelUpdater as modelUpdater


In [114]:
notes = ['C#4', 'D4', 'E4']
map(lambda x: re.sub('\d+', '', x)

SyntaxError: unexpected EOF while parsing (<ipython-input-114-fc26edd09566>, line 2)

In [ ]:
test = [1,2,3,4,5]
k = 3
test[1:3]

In [134]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(fitness)
importlib.reload(constants)
importlib.reload(util)
importlib.reload(individual)
importlib.reload(fitness)
importlib.reload(modelUpdater)

<module 'ea.modelUpdater' from 'B:\\School\\Master\\Thesis\\MasterThesis\\EvoMusicCompanion\\ea\\modelUpdater.py'>

In [ ]:
from ea.individual import Note, Measure, Individual

In [ ]:
len(scores)

## Get corpus

In [ ]:
coreCorpus = music21.corpus.corpora.CoreCorpus()
curr_corpus = music21.corpus.search('mozart', 'composer')

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
lakh_corpus = corpus.corpora.LocalCorpus('lakh')
curr_corpus = lakh_corpus.metadataBundle[0:1006]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [120]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
curr_corpus = wiki_corpus.metadataBundle[0:20]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [ ]:
dMaj = chord.Chord(['D', 'F#', 'A'])
dMaj[0]

In [ ]:
individuals = []

for s in scores[0:10]:
    for p in s.parts:
        measures = p.getElementsByClass(stream.Measure)
        new_measures = []
        counter = 1
        for m in range(len(measures)):
            notes = []
            new_measure = individual.Measure([], 0, [])
            m_notes = measures[m].notesAndRests
            for n in m_notes:
                if n.isRest:
                    if n.duration.type == 'whole':
                        continue
                    pitch_name = 'REST'
                elif n.isChord:
                    continue
                    root = n.root()
                    if root is None:
                        pitch_name = n[0].nameWithOctave
                        continue
                    pitch_name = n.root().nameWithOctave
                else:
                    pitch_name = n.nameWithOctave
                    if '-' in pitch_name or '##' in pitch_name:
                        pitch_name = n.pitch.getEnharmonic().nameWithOctave
                dur = duration.Duration(n.duration.type, None)
                new_note = individual.Note(pitch_name, dur)
                new_measure.notes.append(new_note)
            if len(new_measure.notes) == 0:
                continue
            new_measures.append(new_measure)
    indiv = individual.Individual(new_measures, 0.0)
    individuals.append(indiv)
individuals = initialisation.set_chords_for_population(individuals)
fitness.set_fitness_for_population(individuals)

In [ ]:
individuals[0]

In [139]:
musicPlayer.play_music_xml([individuals[7]])

In [137]:
musicPlayer.play([individuals[7]])

In [136]:
for i in range(len(individuals)):
    print(f'Score {i}: {curr_corpus[i]}')
    fitness.print_fitness_values(individuals[i])
    print('-----------------')

Score 0: <music21.metadata.bundles.MetadataEntry: B:\_School_Master_Thesis_MasterThesis_EvoMusicCompanion_midi_wikifonia_(Unknown) Arrangement Larry Johnson 2011 - I'm Too Far From Where I Started_mxl>
0
Interval pattern fitness: 0.24013157894736842
Duration pattern fitness: 2.746031746031746
Chord tone beat fitness: 0.10526315789473684
Chord tone fitness: 0.5359649122807018
Last note fitness: 1.0
Long note fitness: 0.017543859649122816
Interval size fitness: 0.0
Consecutive rests fitness: -0.05263157894736842
Interval resolution fitness: 0.0
Total fitness: 4.592303675856307
-----------------
Score 1: <music21.metadata.bundles.MetadataEntry: B:\_School_Master_Thesis_MasterThesis_EvoMusicCompanion_midi_wikifonia_& Nobel Prize Winner), Charles Gate Dawes (V_P_ of U_S_A_, Carl Sigman - It's All In The Game_mxl>
-1.0
Interval pattern fitness: 0.2537878787878788
Duration pattern fitness: 1.9807692307692308
Chord tone beat fitness: -0.06060606060606061
Chord tone fitness: 0.5095238095238095


In [ ]:
score = music21.corpus.parse('joplin/maple_leaf_rag.mxl')
scores = util.transpose_piece(score, 'C')
scores = [scores]

In [ ]:
scores[0].show('musicxml')

In [ ]:
dur_matrix = modelTrainer.train_duration_matrix(scores)
pitch_matrix = modelTrainer.train_pitch_matrix(scores);

In [ ]:
# Interval matrix
first_notes_per_bar = []
all_intervals = []
for s in scores:
    noteIterator = s.parts[0].getElementsByClass(stream.Measure)
    if len(noteIterator) == 0:
        noteIterator = s.parts[0].notesAndRests.stream()
    for i in noteIterator:
        firstNote = None
        counter = 1
        m = []
        for j in i.notesAndRests:
            if j.isChord:
                curr_note = note.Note(j.root())
            elif j.isRest:
                m.append('REST')
                continue
            else:
                curr_note = j
            if counter == 1:
                if j.isChord:
                    firstNote = note.Note(j.root())
                else:
                    firstNote = j
                pitch_name = firstNote.nameWithOctave
                if '-' in pitch_name or '##' in pitch_name:
                    pitch_name = firstNote.pitch.getEnharmonic().nameWithOctave
                if pitch_name in NOTE_RANGE:
                    first_notes_per_bar.append(pitch_name)
                counter += 1
                continue
            #print(j.beatStr)
            octave = curr_note.octave
            root = note.Note('C')
            root.octave = octave
            interv = interval.Interval(root,  curr_note).name
            m.append(interv)
            counter += 1
            all_intervals.append(interv)
all_intervals

bigrams = list(nltk.bigrams(all_intervals))
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for i1,i2 in bigrams:
    matrix[i1][i2] += 1
    
for i1 in matrix:
    total = float(sum(matrix[i1].values()))
    for i2 in matrix[i1]:
        matrix[i1][i2] /= total
    
int_matrix = pd.DataFrame(matrix)
int_matrix = int_matrix.fillna(0)


In [ ]:
init_note_vector = defaultdict(lambda: 0)
for i in first_notes_per_bar:
    init_note_vector[i] += 1
total = float(sum(init_note_vector.values()))
for i in init_note_vector:
    init_note_vector[i] /= total
init_note_vector = pd.Series(init_note_vector)

In [ ]:
sim = simulation.Simulation(0.5, 100)
population = sim.run_interactively()

In [ ]:
sim.run(20, None, None)

In [115]:
sim.pitch_matrix

NameError: name 'sim' is not defined

In [ ]:
chord.Chord(['C3', 'E3', 'G3'], quarterlength='4')

In [ ]:
musicPlayer.play(population[0:4])

In [ ]:
selection = [population[0], population[1]]
selection

In [ ]:
selected_population_notes = constants.flatten(list(map(lambda x: x.notes, selection)))
selected_population_pitches = list(map(lambda x: x.pitch, constants.flatten(selected_population_notes)))
selected_population_pitches

In [ ]:
population = sim.update(selection=selection)

In [ ]:
[i for sublist in [[[n.pitch for n in m.notes] for m in p.measures] for p in population[0:1]] for i in sublist]

for p in population:
    for m in p.measures:
        for n in p.notes:
            print(n.pitch)

In [ ]:
[population[1].measures[2].notes, list(map(lambda x: x.notes, population[5].measures[1:3]))]

In [ ]:
musicPlayer.play(population[0:1])

In [ ]:
selection = population[0:2]
selection[0].measures[0]

In [ ]:
modelUpdater.update_matrix(selection, pitch_matrix)

In [ ]:

testIndividual = Individual(
    [
        Measure(
            [
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5))
            ], 0, None),
        Measure(
            [
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5)),
                Note('C#5', Duration('half', 0.5))
            ], 0, None),
        0
    ], 0
)


In [ ]:
population[0].measures[0].notes

In [ ]:
musicPlayer.play_measure(testIndividual.measures[0])

In [ ]:
m = testIndividual.measures[0]

for j in m.notes:
    print(j)

In [ ]:
def convert_measure_to_music21_measure(measure: Measure):
    measure.notes: [Note]
    measure = stream.Measure(1)
    for j in measure.notes:
        if j.pitch == 'REST':
            n = note.Rest()
            n.duration = duration.Duration(quarterLength=j.duration.duration_value / 0.25)
        else:
            n = note.Note(j.pitch)
            n.duration = duration.Duration(quarterLength=j.duration.duration_value / 0.25)
        print(j)
        measure.append(n)

    return measure


In [ ]:
testIndividual.measures[0].notes[0].duration_value

In [ ]:
for i in range(1):
    print(i)